<h1>Содержание<span class="tocSkip"></span></h1>
<br>
<div class="toc">
    <ul class="toc-item">
        <li>
            <span>
                <a href="#1.-Импорт-библиотек">
                    <span class="toc-item-num">1.&nbsp;&nbsp;</span>
                    Импорт библиотек
                </a>
            </span>
        </li>
        <li>
            <span>
                <a href="#2.-Подготовка-данных">
                    <span class="toc-item-num">2.&nbsp;&nbsp;</span>
                    Подготовка данных
                </a>
            </span>
        </li>
        <li>
            <span>
                <a href="#3.-Алгоритмы-градиентного-спуска">
                    <span class="toc-item-num">3.&nbsp;&nbsp;</span>
                    Алгоритмы градиентного спуска
                </a>
            </span>
        </li>
        <li>
            <span>
                <a href="#4.-Алгоритмы-оптимизации">
                    <span class="toc-item-num">4.&nbsp;&nbsp;</span>
                    Алгоритмы оптимизации
                </a>
            </span>
        </li>
        <li>
            <span>
                <a href="#5.-Общий-вывод">
                    <span class="toc-item-num">5.&nbsp;&nbsp;</span>
                    Общий вывод
                </a>
            </span>
        </li>
    </ul>
</div>

# Лабораторная работа №3: Numba

**Задача:** ускорить алгоритмы стохастического градиентного спуска, а также адаптивной и моментной его модификаций с помощью `numba`.

**Источники данных:** набор данных с платформы [Kaggle](https://www.kaggle.com/datasets/rounakbanik/pokemon).

**Описание данных:** набор данных содержит информацию о семи поколениях Покемонов.

---

Для реализации поставленных задач из набора данных `datasets/pokemon.csv` будут взяты следующие столбцы:

* `defense` - признак объектов
* `attack` - целевой признак

<div style="height: 2px; background-color: blue; margin: 10px 0;"></div>

## 1. Импорт библиотек

Импорт всех необходимых библиотек:

In [1]:
import numpy as np
import pandas as pd
import random
import timeit

from numba import njit

from typing import Tuple, Mapping

from sklearn.preprocessing import StandardScaler

from utils.sgd_functions import *

* Внутрипроектный модуль `sgd_function` включает в себя все функции и классы градиентного спуска, использованные в ходе работы над лабораторной работой №1

<div style="height: 2px; background-color: blue; margin: 10px 0;"></div>

## 2. Подготовка данных 

#### Набор данных Pokemon

Сохранение набора данных:

In [2]:
data = (pd.read_csv('datasets/pokemon.csv', usecols=['defense', 'attack'], index_col=0)
          .reset_index())[['defense', 'attack']]

Сохранение значений для `Х` и `Y` из набора данных:

In [3]:
x_pokemon = StandardScaler().fit_transform(data[['defense']]).flatten()
y_pokemon = data['attack'].to_numpy()

Сохранение значений `X` с дополнительным столбцом из единиц:

In [4]:
X_pokemon_ones = np.hstack(((np.ones((x_pokemon.shape[0], 1))), x_pokemon[:, None]))

#### Сгенерированные точки

Задание множества точек:

In [5]:
x_linear = np.linspace(-5, 5, 1000)
y_linear = np.linspace(-5, 5, 1000)

Сохранение значений `X` с дополнительным столбцом из единиц:

In [6]:
X_linear_ones = np.hstack(((np.ones((x_linear.shape[0], 1))), x_linear[:, None]))

>**Вывод**
>
>Данные подготовлены для реализации оптимизации алгоритмов.

<div style="height: 2px; background-color: blue; margin: 10px 0;"></div>

## 3. Алгоритмы градиентного спуска

Функционал `numba` имеет несколько ограничений, требующих внесения изменений в работу исходных алгоритмов, реализованных в Лабораторной работе №1:

* Библиотека поддерживает функционал классов только частично, поэтому исходный класс будет разбит на функции.

* Библиотека не распознаёт переданные в качестве опциональных аргументов значения типа `None`, поэтому будет произведена реализация исходного алгоритма для набора данных и заданной функции отдельно.

* Библиотека не распознаёт переданные в качестве опциональных аргументов функции, поэтому функции будут заданы в теле вычисления градиентов.

<div style="height: 2px; background-color: blue; opacity: 0.2; margin: 10px 0;"></div>

Задание функции, формирующей датафрейм:

In [7]:
def data_framed(data: dict,
                columns: list = ['time']):
    
    return pd.DataFrame(data.keys(),
                        columns=['func']).join(pd.DataFrame(data.values(),
                                               columns=columns))

Задание функции, строящей сводную таблицу по полученным значениям:

In [8]:
def data_joined(data: pd.DataFrame, 
                value: str = 'initial',
                lsuffix: str = '_numba', 
                rsuffix: str = '_initial'):
    
    return data[data['func'].str.contains(value) == False] \
               .reset_index(drop=True) \
               .join(data[data['func'].str.contains(value) == True] \
                         .drop('func', axis=1)
                         .reset_index(drop=True), 
                     lsuffix=lsuffix,
                     rsuffix=rsuffix)

Задание функции, умножающей матрицу на вектор:

In [9]:
@njit
def matmul_vec(X: np.array, 
               vec: np.array) -> np.array:
    
    """Mathematical multiplication
       
       Args: 
           X (np.array): features matrix
           vec (np.array): vector
       
       Return:
           print estimated time
    """
    
    new_X = []
    
    for i in range(X.shape[0]):
        new_value = 0
        
        for j in range(vec.shape[0]):
            new_value += X[i][j] * vec[j]
            
        new_X.append(new_value)
    
    return np.array(new_X)

Задание словаря-хранилища результатов вычисления времени работы функций:

In [10]:
time_comparison_pokemon = {}

Задание словаря-хранилища результатов вычисления времени работы инициализации классов:

In [11]:
time_class = {}

Задание функции сохранения времени работы функций:

In [12]:
def save_time(func: Mapping,
              title: str,
              store: dict = time_comparison_pokemon,
              experiments: int = 100):
    
    """Save time to compare
       
       Args: 
           func (Mapping): estimated function
           title (str): title of a marked time
           store (dict): dictionary to store the results. Defaults to time_comparison
           experiments (int, optional): quantity of experiments. Defaults to 100
       
       Return:
           save checked time (seconds) to a dictionary
    """
    
    results = []
    
    for _ in range(experiments):
        start_time = timeit.default_timer()
        func
        results.append(timeit.default_timer() - start_time)
        
    store[title] = sum(results) / len(results)

Задание начальных весов:

In [13]:
start_w = np.array([-3, 4])

<div style="height: 2px; background-color: blue; opacity: 0.2; margin: 10px 0;"></div>

**Градиент функции потерь**

Задание функции, вычисляющей градиент функции потерь:

In [14]:
@njit
def mse_grad_numba(X: pd.Series, 
                   Y: pd.Series, 
                   w: np.array) -> np.array:
    
    """Calculate mean squared error gradient for dataset"""
    
    y_hat = matmul_vec(X, w)
    error = matmul_vec(X.T, y_hat - Y)
    grad = (2 / X.shape[0]) * error
    
    return grad

Вычисление времени работы с применением `numba`:

In [15]:
save_time(mse_grad_numba(X_pokemon_ones, y_pokemon, start_w), 
          'mse_grad')

Вычисление времени работы исходной функции:

In [16]:
save_time(mse_grad(X_pokemon_ones, y_pokemon, start_w), 
          'mse_grad_initial')

<div style="height: 2px; background-color: blue; opacity: 0.2; margin: 10px 0;"></div>

Вычисление времени работы создания экземпляра исходного класса для данных:

In [17]:
save_time(GradientDescent(x_pokemon, y_pokemon, None, mse_grad), 
          'gd_class', 
          time_class)

In [18]:
gd_pokemon = GradientDescent(x_pokemon, y_pokemon, None, mse_grad)

<div style="height: 2px; background-color: blue; opacity: 0.2; margin: 10px 0;"></div>

**Градиентный спуск**

Задание функции, вычисляющей градиенты:

In [19]:
@njit
def get_gradients(X_ones, Y, w_old, learn_r) -> Tuple:
    """Calculate gradients per each loop saving the result"""
    
    grad = mse_grad_numba(X_ones, Y, w_old)

    dw = learn_r * grad
    w_new = w_old - dw

    return w_new, grad, dw

Вычисление времени работы с применением `numba`:

In [20]:
save_time(get_gradients(X_pokemon_ones, y_pokemon, start_w, 0.01), 
          'gradient_descent')

Вычисление времени работы исходной функции:

In [21]:
save_time(gd_pokemon.get_gradients(X_pokemon_ones, x_pokemon, y_pokemon, start_w, 'gradient', 0.01), 
          'gradient_descent_initial')

<div style="height: 2px; background-color: blue; opacity: 0.2; margin: 10px 0;"></div>

**Стохастический градиентный спуск**

Задание функции, вычисляющей стохастический градиентный спуск:

In [22]:
def stochastic_descent(X_ones, Y, w_old, learn_r) -> Tuple:
    """Stochastic gradient descent algorithm"""

    @njit
    def get_random(X_ones):
        return np.random.randint(X_ones.shape[0])
    
    i = get_random(X_ones)

    return get_gradients(X_ones[i, None], Y[i, None], w_old, learn_r)

Вычисление времени работы с применением `numba`:

In [23]:
save_time(stochastic_descent(X_pokemon_ones, y_pokemon, start_w, 0.01), 
          'stochastic_descent')

Вычисление времени работы исходной функции:

In [24]:
save_time(gd_pokemon.stochastic_descent(start_w, 'stochastic', 0.01), 
          'stochastic_descent_initial')

<div style="height: 2px; background-color: blue; opacity: 0.2; margin: 10px 0;"></div>

**Стохастический градиентный спуск по мини-батчам**

Задание функции, вычисляющей градиентный спуск по мини-батчам:

In [25]:
@njit
def minibatch_descent(X_ones, Y, w_old, learn_r, batch_size) -> Tuple:
    """Stochastic mini-batch gradient descent algorithm"""
    
    w_new_list, grad_list = [], []

    batches_count = X_ones.shape[0] // batch_size

    for i in range(batches_count):
        begin = i * batch_size
        end = (i + 1) * batch_size

        w_new, grad, dw = get_gradients(X_ones[begin:end, :], Y[begin:end], w_old, learn_r)
        
        w_new_list.append(w_new)
        grad_list.append(dw)
    
    return w_new_list, grad_list, dw

Вычисление времени работы с применением `numba`:

In [26]:
save_time(minibatch_descent(X_pokemon_ones, y_pokemon, start_w, 0.01, 10), 
          'minibatch_descent')

Вычисление времени работы исходной функции:

In [27]:
save_time(gd_pokemon.minibatch_descent(start_w, 'minibatch', 0.01, 10), 
          'minibatch_descent_initial')

<div style="height: 2px; background-color: blue; opacity: 0.2; margin: 10px 0;"></div>

Задание функции, определяющей тип градиентного спуска:

In [28]:
def grad_condition(X_ones, Y, w_old, descent, learn_r, batch_size) -> Tuple:
    """Gradient type"""

    if descent == 'gradient':
        w_new, grads, dw = get_gradients(X_ones, Y, w_old, learn_r)

    elif descent == 'stochastic':
        w_new, grads, dw = stochastic_descent(X_ones, Y, w_old, learn_r)

    elif descent == 'minibatch':
        w_new, grads, dw = minibatch_descent(X_ones, Y, w_old, learn_r, batch_size)

    else:
        raise Exception('Incorrect type of descent')
    
    return w_new, grads, dw

<div style="height: 2px; background-color: blue; opacity: 0.2; margin: 10px 0;"></div>

Задание функции, вычисляющей градиентный спуск:

In [29]:
def descent(X: np.array,
            Y: np.array,
            w0: np.array,
            descent: str,    
            learn_r: float = 0.01,
            max_iters: int = 300,
            batch_size: int = None,
            epsilon: int = 2e-4,
            threshold: int = 2e-4,
            seed_v: int = 2020) -> np.array:

    """Gradient descent calculation"""
    
    @njit
    def seed(seed_v: int = seed_v):
        return np.random.seed(seed_v)
    
    seed()
    
    weights, grads = [], []

    w_old = w0
    iters = 1
    dw = np.array(2 * epsilon)
    
    X_ones = np.hstack(((np.ones((X.shape[0], 1))), X[:, None]))

    while abs(dw).sum() > threshold and iters <= max_iters:
        w_new, grad, dw = grad_condition(X_ones, Y, w_old, 
                                         descent, learn_r, batch_size)
        
        if len(w_new) > 2:
            weights.extend(w_new)
            grads.extend(grad)
            
            w_old = w_new[-1]
            
        else:
            weights.append(w_new)
            grads.append(grad)
            
            w_old = w_new
        
        iters += 1
    
    return np.array([weights, grads])

Задание начальных весов:

In [30]:
gd_pokemon.start_w = np.array([-20, -10])

---

*Градиентный спуск*

Вычисление времени работы с применением `numba`:

In [31]:
save_time(descent(x_pokemon, y_pokemon, gd_pokemon.start_w, 'gradient', 0.01, 1000), 
          'descent_gradient')

Сохранение результатов:

In [32]:
grads = descent(x_pokemon, y_pokemon, gd_pokemon.start_w, 'gradient', 0.01, 1000)

Вычисление времени работы исходной функции:

In [33]:
save_time(gd_pokemon.descent(gd_pokemon.start_w, 'gradient', 0.01, 1000), 
          'descent_gradient_initial')

Сохранение результатов:

In [34]:
gd_pokemon.grads = gd_pokemon.descent(gd_pokemon.start_w, 'gradient', 0.01, 1000)

---

*Стохастический градиентный спуск*

Вычисление времени работы с применением `numba`:

In [35]:
save_time(descent(x_pokemon, y_pokemon, gd_pokemon.start_w, 'stochastic', 0.01, 1000), 
          'descent_stochastic')

Сохранение результатов:

In [36]:
grads_stochastic = descent(x_pokemon, y_pokemon, gd_pokemon.start_w, 'stochastic', 0.01, 1000)

Вычисление времени работы исходной функции:

In [37]:
save_time(gd_pokemon.descent(gd_pokemon.start_w, 'stochastic', 0.01, 1000), 
          'descent_stochastic_initial')

Сохранение результатов:

In [38]:
gd_pokemon.grads_stochastic = gd_pokemon.descent(gd_pokemon.start_w, 'stochastic', 0.01, 1000)

---

*Стохастический градиентный спуск по мини-батчам*

Вычисление времени работы с применением `numba`:

In [39]:
save_time(descent(x_pokemon, y_pokemon, gd_pokemon.start_w, 'minibatch', 0.001, 5000, 50), 
          'descent_minibatch')

Сохранение результатов:

In [40]:
grads_minibatch = descent(x_pokemon, y_pokemon, gd_pokemon.start_w, 'minibatch', 0.001, 5000, 50)

Вычисление времени работы исходной функции:

In [41]:
save_time(gd_pokemon.descent(gd_pokemon.start_w, 'minibatch', 0.001, 5000, 50), 
          'descent_minibatch_initial')

Сохранение результатов:

In [42]:
gd_pokemon.grads_minibatch = gd_pokemon.descent(gd_pokemon.start_w, 'minibatch', 0.001, 5000, 50)

>**Вывод**
>
>Произведена оптимизация скорости работы алгоритмов градиентного спуска.

<div style="height: 2px; background-color: blue; margin: 10px 0;"></div>

## 4. Алгоритмы оптимизации

Вычисление времени работы создания экземпляра исходного класса для данных:

In [43]:
save_time(Optimization(gd_pokemon.grads['weights'], gd_pokemon.grads['grads']), 
          'optim_class_grads', 
          time_class)

In [44]:
gd_pokemon.optimizer_grads = Optimization(gd_pokemon.grads['weights'],
                                          gd_pokemon.grads['grads'])

---

Вычисление времени работы создания экземпляра исходного класса для данных:

In [45]:
save_time(Optimization(gd_pokemon.grads_stochastic['weights'], gd_pokemon.grads_stochastic['grads']), 
          'optim_class_stochastic', 
          time_class)

In [46]:
gd_pokemon.optimizer_stochastic = Optimization(gd_pokemon.grads_stochastic['weights'],
                                               gd_pokemon.grads_stochastic['grads'])

---

Вычисление времени работы создания экземпляра исходного класса для данных:

In [47]:
save_time(Optimization(gd_pokemon.grads_minibatch['weights'], gd_pokemon.grads_minibatch['grads']), 
          'optim_class_minibatch', 
          time_class)

In [48]:
gd_pokemon.optimizer_minibatch = Optimization(gd_pokemon.grads_minibatch['weights'],
                                              gd_pokemon.grads_minibatch['grads'])

<div style="height: 2px; background-color: blue; opacity: 0.2; margin: 10px 0;"></div>

**ADAM**

Задание функции оптимизации ADAM:

In [49]:
@njit
def adam_optimizer(weights: np.array, 
                   grads: np.array, 
                   learn_r: float = 0.01,
                   beta1: float = 0.9, 
                   beta2: float = 0.999,
                   epsilon: int = 2e-4) -> np.array:
    
    """Calculate ADAM optimization"""
    
    results = []
    
    m = [np.array([0.0, 0.0])] * weights.shape[0]
    v = [np.array([0.0, 0.0])] * weights.shape[0]
    
    iters = 1

    while iters <= weights.shape[0]:
        i = iters - 1
        m[i] = beta1 * m[i] + (1 - beta1) * grads[i]          
        v[i] = beta2 * v[i] + (1 - beta2) * grads[i] ** 2
        
        m_corrected = m[i] / (1 - beta1 ** iters)
        v_corrected = v[i] / (1 - beta2 ** iters)

        new_weight = weights[i] - learn_r * m_corrected / (np.sqrt(v_corrected) + epsilon)
        results.append(new_weight)
        
        iters += 1        
    
    return results

*Градиентный спуск*

Вычисление времени работы с применением `numba`:

In [50]:
save_time(adam_optimizer(grads[0], grads[1], 0.001), 
          'adam_gradient')

Вычисление времени работы исходной функции:

In [51]:
save_time(gd_pokemon.optimizer_grads.optimize('adam', 0.001), 
          'adam_gradient_initial')

---

*Стохастический градиентный спуск*

Вычисление времени работы с применением `numba`:

In [52]:
save_time(adam_optimizer(grads_stochastic[0], grads_stochastic[1], 0.001), 
          'adam_stochastic')

Вычисление времени работы исходной функции:

In [53]:
save_time(gd_pokemon.optimizer_stochastic.optimize('adam', 0.001), 
          'adam_stochastic_initial')

---

*Стохастический градиентный спуск по мини-батчам*

Вычисление времени работы с применением `numba`:

In [54]:
save_time(adam_optimizer(grads_minibatch[0], grads_minibatch[1], 0.001), 
          'adam_minibatch')

Вычисление времени работы исходной функции:

In [55]:
save_time(gd_pokemon.optimizer_minibatch.optimize('adam', 0.001), 
          'adam_minibatch_initial')

<div style="height: 2px; background-color: blue; opacity: 0.2; margin: 10px 0;"></div>

**Метод моментов**

Задание функции оптимизации моментов:

In [56]:
@njit
def momentum_optimizer(weights: np.array, 
                       grads: np.array,
                       learn_r: float = 0.01,
                       epsilon: int = 2e-4) -> np.array:
    
    """Calculate momentum optimization"""
    
    results = []

    previous_updates = [0] * weights.shape[0]
    prevs = []
    mom_coeff = 1

    for weight, grad, prev_update in zip(weights, grads, previous_updates):
        delta = learn_r * grad - mom_coeff * prev_update
        new_weight = weight - delta

        prevs.append(delta)
        results.append(new_weight)
        previous_updates = prevs  

    return results

*Градиентный спуск*

Вычисление времени работы с применением `numba`:

In [57]:
save_time(momentum_optimizer(grads[0], grads[1], 0.001), 
          'momentum_gradient')

Вычисление времени работы исходной функции:

In [58]:
save_time(gd_pokemon.optimizer_grads.optimize('momentum', 0.001), 
          'momentum_gradient_initial')

---

*Стохастический градиентный спуск*

Вычисление времени работы с применением `numba`:

In [59]:
save_time(momentum_optimizer(grads_stochastic[0], grads_stochastic[1], 0.001), 
          'momentum_stochastic')

Вычисление времени работы исходной функции:

In [60]:
save_time(gd_pokemon.optimizer_stochastic.optimize('momentum', 0.001), 
          'momentum_stochastic_initial')

---

*Стохастический градиентный спуск по мини-батчам*

Вычисление времени работы с применением `numba`:

In [61]:
save_time(momentum_optimizer(grads_minibatch[0], grads_minibatch[1], 0.001), 
          'momentum_minibatch')

Вычисление времени работы исходной функции:

In [62]:
save_time(gd_pokemon.optimizer_minibatch.optimize('momentum', 0.001), 
          'momentum_minibatch_initial')

>**Вывод**
>
>Произведена оптимизация скорости работы алгоритмов оптимтзации.

<div style="height: 2px; background-color: blue; margin: 10px 0;"></div>

## 5. Общий вывод

Для большей чистоты эксперимента было взято усреднённое значение 100 итераций замера времени, произведённых по каждой функции. Замер производился в секундах.

Выведение на экран сводной таблицы полученных вычислений градиентного спуска:

In [63]:
data_joined(data_framed(time_comparison_pokemon))[:7]

,func,time_numba,time_initial
0,mse_grad,1.150000e-07,1.140000e-07
1,gradient_descent,1.110000e-07,1.080000e-07
2,stochastic_descent,1.130000e-07,1.160000e-07
3,minibatch_descent,1.110000e-07,1.140000e-07
4,descent_gradient,1.190000e-07,1.230000e-07
5,descent_stochastic,1.120000e-07,1.650000e-07
6,descent_minibatch,1.160000e-07,1.420000e-07


Выведение на экран вычислений формирования класса:

In [64]:
data_framed(time_class)[:1]

,func,time
0,gd_class,1.070000e-07


>**Вывод**
>
>Ускоренные с помощью `numba` функции показали лучшее время работы по показателям скорости, чем исходные. В целом результаты находятся в одном диапазоне, однако заметно значительное ускорение работы некоторых функций относительно исходного набора данных.

Выведение на экран сводной таблицы полученных вычислений алгоритмов оптимизации:

In [65]:
data_joined(data_framed(time_comparison_pokemon))[7:].reset_index(drop=True)

,func,time_numba,time_initial
0,adam_gradient,1.100000e-07,1.270000e-07
1,adam_stochastic,1.230000e-07,1.560000e-07
2,adam_minibatch,1.210000e-07,1.230000e-07
3,momentum_gradient,1.120000e-07,1.270000e-07
4,momentum_stochastic,1.640000e-07,1.180000e-07
5,momentum_minibatch,1.200000e-07,1.130000e-07


Выведение на экран вычислений формирования класса:

In [66]:
data_framed(time_class)[1:]

,func,time
1,optim_class_grads,1.210000e-07
2,optim_class_stochastic,1.180000e-07
3,optim_class_minibatch,1.210000e-07


>**Вывод**
>
>Ускоренный с помощью `numba` алгоритм оптимизации ADAM показал хорошую скорость выполнения относительно показателей скорости исходного алгоритма. Однако результаты моментного метода по одному из показателей проявили себя хуже, наоборот увеличив время выполнения.

<div style="text-align: center; font-size: 20px; padding: 15px 0;">
    <a href="#Содержание" data-toc-modified-id="Содержание" style="text-decoration: none; color: #296eaa; border: 2px dashed #296eaa; opacity: 0.8; border-radius: 3px; padding: 10px 80px;">
        Наверх к содержанию ↑
    </a>
</div>